## Librerias

In [2]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import category_encoders as ce
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.impute import SimpleImputer
import seaborn as sns
from scipy.stats import chi2_contingency
import scipy.stats as ss
import os
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer

import warnings
warnings.filterwarnings('ignore')


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Funciones

In [4]:
def agrupar_categorias_pequenas(df, columna, umbral=100):
    # Contar la frecuencia de cada categoría
    conteo_categorias = df[columna].value_counts()
    
    # Identificar categorías con observaciones por debajo del umbral
    categorias_pequenas = conteo_categorias[conteo_categorias < umbral].index
    
    # Mantener el mismo tipo de las categorías originales
    tipo_original = df[columna].dtype.type
    
    # Reemplazar esas categorías con 'Otros', asegurando el mismo tipo de dato
    df[columna] = df[columna].apply(lambda x: tipo_original('Otros') if x in categorias_pequenas else x)
    
    return df

def cramers_v(confusion_matrix):
    chi2 = ss.chi2_contingency(confusion_matrix, correction=False)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    if min(r, k) == 1:
        return np.nan  # Evita dividir por cero
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))

def expand_list(df, column):
    # Crear un conjunto de categorías
    categories = set()
    
    # Recorrer todas las listas y agregar cada categoría única al conjunto
    for row in df[column]:
        if pd.notna(row):
            # Verificar si el valor es una lista; si es string, se convierte en lista
            if isinstance(row, str):
                try:
                    row_list = eval(row)  # Transforma la string en lista
                except:
                    row_list = [row]  # Si no es lista, lo trata como valor único
            else:
                row_list = row if isinstance(row, list) else [row]
            
            # Añadir los elementos de la lista a las categorías
            categories.update(map(str, row_list))  # Convertir todos los elementos a string

    # Crear un DataFrame temporal para almacenar las nuevas columnas
    new_columns = pd.DataFrame(index=df.index)

    # Llenar el DataFrame temporal con las nuevas columnas
    for category in categories:
        # Crear el nombre de la nueva columna con el sufijo del nombre de la columna original
        new_column_name = f"{category}_{column}"
        new_columns[new_column_name] = df[column].apply(lambda x: 1 if pd.notna(x) and str(category) in str(x) else 0)

    # Concatenar el nuevo DataFrame con las columnas originales
    df = pd.concat([df.drop(columns=[column]), new_columns], axis=1)

    return categories, df


def categorizar_hora(hora):
        hora = int(hora.split(':')[0])
        if 0 <= hora < 12:
            return 'mañana'
        elif 12 <= hora < 18:
            return 'tarde'
        else:
            return 'noche'

def convertir_auction_time(df):
    
    df['auction_time'] = pd.to_datetime(df['auction_time'], unit='s').dt.strftime('%H:%M:%S')
    
    # Crear una nueva columna con las categorías de tiempo
    df['parte_del_dia'] = df['auction_time'].apply(categorizar_hora)
    
    # Crear columnas binarias para cada parte del día
    df['mañana'] = (df['parte_del_dia'] == 'mañana').astype(int)
    df['tarde'] = (df['parte_del_dia'] == 'tarde').astype(int)
    df['noche'] = (df['parte_del_dia'] == 'noche').astype(int)
    
    # Opcional: Eliminar la columna 'parte_del_dia' si no se necesita
    df.drop(['parte_del_dia', 'auction_time'], axis=1, inplace=True)

    return ['mañana', 'tarde', 'noche'], df

def imputar_nan_como_otros(df):
    for columna in df.columns:
        if df[columna].isnull().any():  # Verificar si hay NaN en la columna
            tipo_original = df[columna].dtype  # Guardar el tipo original de la columna
            # Imputar 'Otros' y asegurarse de que el tipo de dato se mantiene
            df[columna] = df[columna].fillna(tipo_original.type('Otros'))
    return df

def data_cleaning(df):
    all_categories = set()

    categorical_features = [col for col in df.select_dtypes(include=['object']).columns.tolist() if col not in ['auction_list_0', 'action_list_1', 'action_list_2', 'auction_time']]

    categories, df = convertir_auction_time(df)
    all_categories.update(categories)

    categories, df = expand_list(df, 'auction_list_0')
    all_categories.update(categories)

    categories, df = expand_list(df, 'action_list_1')
    all_categories.update(categories)

    categories, df = expand_list(df, 'action_list_2')
    all_categories.update(categories)

    for col in categorical_features:
        df = agrupar_categorias_pequenas(df, col, umbral=100) 

    return all_categories, df

def ajustar_columnas_test(train_df, test_df):
    """
    Asegura que las columnas del conjunto de test coincidan con las del conjunto de entrenamiento,
    con la excepción de que 'Label' no se puede borrar de train y 'id' no se puede borrar de test.
    
    Si faltan columnas en el conjunto de test, se agregan con valor 0.
    Si sobran columnas en el conjunto de test que no están en el de entrenamiento, se eliminan.
    
    Parameters:
    - train_df: DataFrame del conjunto de entrenamiento (con las columnas transformadas)
    - test_df: DataFrame del conjunto de test (sin transformar)
    
    Returns:
    - test_df: DataFrame del conjunto de test con las columnas ajustadas.
    """
    # Encontrar las columnas que están en train pero no en test, excepto 'Label'
    missing_cols = set(train_df.columns) - set(test_df.columns) - {'Label'}

    # Encontrar las columnas que están en test pero no en train, excepto 'id'
    extra_cols = set(test_df.columns) - set(train_df.columns) - {'id'}

    # Agregar las columnas faltantes en el conjunto de test con valor 0
    for col in missing_cols:
        test_df[col] = 0

    # Eliminar las columnas extra del conjunto de test, excepto 'id'
    test_df = test_df.drop(columns=extra_cols)

    # Asegurarse de que las columnas estén en el mismo orden que en el conjunto de entrenamiento
    test_df = test_df[train_df.columns.difference(['Label']).tolist() + ['id']]

    return test_df



## Data

In [10]:
# train_data_21 = pd.read_csv("data/ctr_21.csv")
# train_data_20 = pd.read_csv("data/ctr_20.csv")
# train_data_19 = pd.read_csv("data/ctr_19.csv")
# train_data_18 = pd.read_csv("data/ctr_18.csv")
# train_data_17 = pd.read_csv("data/ctr_17.csv")
# train_data_16 = pd.read_csv("data/ctr_16.csv")
# train_data_15 = pd.read_csv("data/ctr_15.csv")
# train_data_combined = pd.concat([train_data_21, train_data_20, train_data_19, train_data_18, train_data_17, train_data_16, train_data_15], axis=0)
# train_data_combined = pd.concat([train_data_21, train_data_20], axis=0)
train_data_combined = pd.read_csv("data/ctr_21.csv")


In [11]:
random_state = 43992294

In [12]:
# Reducir el conjunto de datos mientras se mantiene la distribución de 'label'
train_data_combined, _ = train_test_split(train_data_combined, train_size=100000, stratify=train_data_combined['Label'], random_state=42)

# Verificar la distribución en el conjunto reducido
print(train_data_combined['Label'].value_counts(normalize=True))

Label
0    0.98567
1    0.01433
Name: proportion, dtype: float64


## Caracteristicas principales

In [ ]:
# Imprimir la cantidad de filas del dataset combinado
print(f"Cantidad de filas en el dataset combinado: {train_data_combined.shape[0]}")

# Ver porcentaje de clics vs no clics en la columna Label
label_counts = train_data_combined['Label'].value_counts(normalize=True) * 100
print("\nPorcentaje de clics (1) y no clics (0):")
print(label_counts)

# Cantidad de clics (1) y no clics (0)
label_counts_abs = train_data_combined['Label'].value_counts()
print("\nCantidad de clics (1) y no clics (0):")
print(label_counts_abs)

In [ ]:
# Resumen estadístico de las características numéricas
print(train_data_combined.describe())

# Resumen de las características categóricas
print(train_data_combined.describe(include='object'))

In [ ]:
# Ver el porcentaje de valores faltantes por columna
missing_data = train_data_combined.isnull().mean() * 100
print(missing_data[missing_data > 0].sort_values(ascending=False))

In [5]:
# Dividir los datos en numéricos y categóricos
numeric_features = train_data_combined.select_dtypes(include=['number']).columns.tolist()

categorical_features = train_data_combined.select_dtypes(include=['object']).columns.tolist()

## Columna por columna

In [ ]:
# Iterar sobre cada columna y mostrar los valores únicos
for column in train_data_combined.columns:
    unique_values = train_data_combined[column].unique()
    print(f"Columna: {column}")
    print(f"Valores únicos ({len(unique_values)}): {unique_values}")
    print("-" * 50)  # Separador entre columnas

In [ ]:
# Iterar sobre cada columna y mostrar los valores únicos y la cantidad de veces que aparecen
for column in train_data_combined.columns:
    print(f"Columna: {column}")
    print(f"Valores únicos y su frecuencia:")
    print(train_data_combined[column].value_counts())
    print("-" * 50)  # Separador entre columnas

In [9]:
# Inicializar un conjunto vacío para almacenar los entity ids únicos
unique_entity_ids_2 = set()

# Recorrer cada fila de la columna 'action_list_2'
for lista in train_data_combined['action_list_2'].dropna():
    # Convertir el string de la lista a una lista real y agregar cada valor al conjunto
    unique_entity_ids_2.update(eval(lista))  # eval transforma la string en lista si es necesario

# Convertir el conjunto en una lista ordenada
unique_entity_ids_2 = sorted(unique_entity_ids_2)

# Mostrar los entity ids únicos
print(unique_entity_ids_2)
print(len(unique_entity_ids_2))



[-7265, -7264, -7263, -7196, -7195, -7190, -7143, -7127, -7126, -7112, -7111, -6946, -6938, -6929, -6905, -6904, -6876, -6875, -6871, -6867, -6866, -6865, -6850, -6849, -6848, -6823, -6800, -6780, -6775, -6774, -6770, -6621, -6620, -6618, -6617, -6615, -6548, -6547, -6544, -6543, -6454, -6309, -6226, -6224, -6223, -6220, -6219, -6218, -6217, -6125, -6119, -6118, -5902, -5613, -5605, -5604, -5603, -5579, -5578, -5577, -5576, -5560, -5559, -5471, -5470, -5469, -2560, 2560, 5469, 5470, 5471, 5559, 5560, 5576, 5577, 5578, 5579, 5603, 5604, 5605, 5613, 5902, 6118, 6119, 6125, 6217, 6218, 6219, 6220, 6223, 6224, 6226, 6309, 6451, 6454, 6543, 6544, 6547, 6548, 6615, 6616, 6618, 6620, 6621, 6770, 6774, 6775, 6780, 6800, 6823, 6848, 6849, 6850, 6865, 6866, 6867, 6871, 6875, 6876, 6904, 6905, 6929, 6938, 6946, 7111, 7112, 7126, 7127, 7143, 7190, 7195, 7196, 7263, 7264, 7265]


In [10]:
# Inicializar un conjunto vacío para almacenar los entity ids únicos
unique_entity_ids = set()

# Recorrer cada fila de la columna 'action_list_1'
for lista in train_data_combined['action_list_1'].dropna():
    # Convertir el string de la lista a una lista real y agregar cada valor al conjunto
    unique_entity_ids.update(eval(lista))  # eval transforma la string en lista si es necesario

# Convertir el conjunto en una lista ordenada
unique_entity_ids = sorted(unique_entity_ids)

# Mostrar los entity ids únicos
print(unique_entity_ids)
print(len(unique_entity_ids))

[-7199, -7195, -7194, -7190, -7033, -6903, -6902, -6874, -6871, -6824, -6823, -6780, -6779, -6618, -6617, -6616, -6615, -6614, -6613, -6454, -6451, -6119, -6118, -5736, -5579, -5578, -5577, -5559, -2606, 2606, 5559, 5577, 5578, 5579, 5736, 6118, 6119, 6451, 6454, 6614, 6615, 6616, 6617, 6618, 6779, 6780, 6823, 6824, 6871, 6874, 6902, 6903, 7033, 7190, 7194, 7195, 7199]


In [18]:
# Calcular la cantidad de valores NaN en cada columna
nan_counts = train_data_combined.isna().sum()

# Filtrar columnas que tienen al menos un NaN
nan_counts_filtered = nan_counts[nan_counts > 0]

# Mostrar las columnas con valores NaN de mayor a menor
print(nan_counts_filtered.sort_values(ascending=False))

creative_categorical_3     1453098
creative_categorical_6     1261556
creative_categorical_7     1256292
creative_categorical_12    1253433
auction_age                1230041
creative_categorical_5     1205582
creative_categorical_2     1197706
gender                     1175827
action_list_1               549228
auction_categorical_9       415685
action_list_2               412296
creative_categorical_4      317639
creative_width              279643
creative_height             279643
auction_categorical_6       119775
auction_boolean_1            44370
auction_boolean_2            39803
auction_boolean_0            22728
auction_list_0               22574
creative_categorical_9       13317
auction_categorical_2        12693
auction_categorical_0         7034
auction_categorical_4         5997
timezone_offset               4885
auction_categorical_3         4870
auction_categorical_11        2396
auction_categorical_7           43
auction_categorical_12           2
dtype: int64


## Data cleaning

In [ ]:
# Iterar sobre cada columna y mostrar los valores únicos
for column in train_data_combined.columns:
    unique_values = train_data_combined[column].unique()
    print(f"Columna: {column}")
    print(f"Valores únicos ({len(unique_values)}): {unique_values}")
    print("-" * 50)  # Separador entre columnas

In [13]:
categories, train_data_combined = data_cleaning(train_data_combined)

/var/folders/5k/nl_6tbmj7mx91g71vzpwrch40000gn/T/ipykernel_67079/2714462713.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[new_column_name] = df[column].apply(lambda x: 1 if pd.notna(x) and str(category) in str(x) else 0)
/var/folders/5k/nl_6tbmj7mx91g71vzpwrch40000gn/T/ipykernel_67079/2714462713.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[new_column_name] = df[column].apply(lambda x: 1 if pd.notna(x) and str(category) in str(x) else 0)
/var/folders/5k/nl_6tbmj7mx91g71vzpwrch40000gn/T/ipyke

In [63]:
# Guardar el dataset modificado como CSV antes de aplicar One-Hot Encoding
train_data_combined.to_csv('train_data_cleaned.csv', index=False)

In [ ]:
train_data_combined.columns.tolist()

In [ ]:
# Iterar sobre cada columna y mostrar los valores únicos
for column in train_data_combined.columns:
    unique_values = train_data_combined[column].unique()  # Obtener los valores únicos para cada columna
    print(f"Columna: {column}")
    print(f"Valores únicos ({len(unique_values)}): {unique_values}")
    print("-" * 50)  # Separador entre columnas

In [ ]:
train_data_combined.isnull().sum()[train_data_combined.isnull().sum() > 0]

# Encodings

## Target Encoding

In [ ]:
# Dividir los datos en entrenamiento y validación con estratificación
X_train, X_val, y_train, y_val = train_test_split(
    train_data_combined.drop(columns='Label'),  # Características
    train_data_combined['Label'],               # Variable objetivo
    test_size=0.2,                              # 20% para validación
    stratify=train_data_combined['Label'],      # Estratificación basada en la variable objetivo
    random_state=random_state                   # Semilla para reproducibilidad
)

# Verificar la proporción de clases en el conjunto de entrenamiento y validación
print("Proporción en el conjunto de entrenamiento:")
print(y_train.value_counts(normalize=True))

print("Proporción en el conjunto de validación:")
print(y_val.value_counts(normalize=True))


In [ ]:
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

# Crear el Target Encoder para las columnas categóricas
target_encoder = ce.TargetEncoder(cols=categorical_features)

# Ajustar y transformar los datos de entrenamiento
X_train_encoded = target_encoder.fit_transform(X_train, y_train)

# Transformar los datos de validación
X_val_encoded = target_encoder.transform(X_val)

# Crear un imputador que llene los valores NaN con la media de cada columna
imputer = SimpleImputer(strategy='mean')

# Aplicar el imputador a los datos de entrenamiento
X_train_encoded = imputer.fit_transform(X_train_encoded)

# También aplica el imputador al conjunto de validación
X_val_encoded = imputer.transform(X_val_encoded)

# Convertir los resultados a DataFrame y asignar las columnas originales
X_train_encoded = pd.DataFrame(X_train_encoded, columns=X_train.columns)
X_val_encoded = pd.DataFrame(X_val_encoded, columns=X_val.columns)

# Ver los primeros datos
print(X_train_encoded.head())
print(X_val_encoded.head())

In [ ]:
# Cargar los datos de testeo
test_data = pd.read_csv("data/ctr_test.csv")

# Eliminar las columnas no deseadas
test_data = test_data.drop(columns=columns_to_drop)

test_data_encoded = target_encoder.transform(test_data.drop(columns=["id"]))
test_data_encoded = imputer.transform(test_data_encoded)  # Imputar los valores faltantes en el set de testeo

# Convertir los resultados a DataFrame y asignar las columnas originales
test_data_encoded = pd.DataFrame(test_data_encoded, columns=test_data.drop(columns=["id"]).columns)

# comparar tamaño de columnas
print(X_train_encoded.columns)
print(test_data_encoded.columns)

## One Hot Encoding

In [6]:
train_data_combined = pd.read_csv('train_data_cleaned.csv')

In [ ]:
train_data_combined.columns.tolist()

In [14]:
# Dividir los datos en entrenamiento y validación con estratificación
X_train, X_val, y_train, y_val = train_test_split(
    train_data_combined.drop(columns='Label'),  # Características
    train_data_combined['Label'],               # Variable objetivo
    test_size=0.2,                              # 20% para validación
    stratify=train_data_combined['Label'],      # Estratificación basada en la variable objetivo
    random_state=random_state                   # Semilla para reproducibilidad
)

# Asegurarse de que X_train y X_val sean dataframes de pandas
X_train = pd.DataFrame(X_train)
X_val = pd.DataFrame(X_val)

# Asegurarse de que y_train y y_val sean series de pandas
y_train = pd.Series(y_train)
y_val = pd.Series(y_val)

In [15]:
# Definir las columnas categóricas y numéricas
categorical_features_to_encode = [col for col in X_train.select_dtypes(include=['object']).columns if col not in categories]
numeric_features = X_train.select_dtypes(include=['number']).columns.tolist()

# Crear el preprocesador para manejar NaNs y aplicar One-Hot Encoding
preprocessor = ColumnTransformer(
    transformers=[
        # Imputar valores numéricos con la media y no modificar las demás columnas
        ('num', SimpleImputer(strategy='mean'), numeric_features),
        
        # Imputar valores categóricos con 'Desconocido' y aplicar One-Hot Encoding
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='Desconocido')),
            ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
        ]), categorical_features_to_encode)
    ],
    remainder='passthrough'  # Mantener el resto de las columnas como están
)

# Aplicar el pipeline de preprocesamiento a los datos de entrenamiento
X_train_encoded = preprocessor.fit_transform(X_train)
# Aplicar el pipeline al conjunto de validación
X_val_encoded = preprocessor.transform(X_val)

# Convertir los resultados a DataFrame si es necesario
X_train_encoded = pd.DataFrame(X_train_encoded, columns=preprocessor.get_feature_names_out())
X_val_encoded = pd.DataFrame(X_val_encoded, columns=preprocessor.get_feature_names_out())

# Asegurarse de que y_train y y_val sean series de pandas
y_train = pd.Series(y_train)
y_val = pd.Series(y_val)

In [16]:
# Paso 1: Cargar los datos de testeo
test_data = pd.read_csv("data/ctr_test.csv")

# Paso 2: Modificar la columna auction_list_0 (expansión de la lista en múltiples columnas)
categories, test_data = data_cleaning(test_data)

/var/folders/5k/nl_6tbmj7mx91g71vzpwrch40000gn/T/ipykernel_67079/2714462713.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[new_column_name] = df[column].apply(lambda x: 1 if pd.notna(x) and str(category) in str(x) else 0)
/var/folders/5k/nl_6tbmj7mx91g71vzpwrch40000gn/T/ipykernel_67079/2714462713.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_columns[new_column_name] = df[column].apply(lambda x: 1 if pd.notna(x) and str(category) in str(x) else 0)
/var/folders/5k/nl_6tbmj7mx91g71vzpwrch40000gn/T/ipyke

In [17]:
# Paso 3: Ajustar las columnas del conjunto de testeo
test_data = ajustar_columnas_test(X_train, test_data)

# Paso 4: Aplicar el preprocesador al conjunto de testeo
test_data_encoded = preprocessor.transform(test_data)

# Convertir los resultados a DataFrame si es necesario
test_data_encoded = pd.DataFrame(test_data_encoded, columns=preprocessor.get_feature_names_out())

print(f"Columnas en el conjunto de entrenamiento: {X_train_encoded.shape[1]}")
print(f"Columnas en el conjunto de testeo: {test_data_encoded.shape[1]}")

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 4, 7, 8, 10, 12, 15, 16, 18, 21, 22, 24, 25, 29, 30, 31, 33, 34, 35, 38] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Columnas en el conjunto de entrenamiento: 1471
Columnas en el conjunto de testeo: 1471


# Busqueda de hiperparametros

## Random Forest

### Random search hiperparametros

In [ ]:
# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'max_samples': np.linspace(0.5, 1.0, num=10),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Definir el modelo
model = RandomForestClassifier(random_state=random_state)

# Definir el RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=50,  # Número de combinaciones de hiperparámetros a probar
    scoring='roc_auc',
    cv=3,  # Número de folds en la validación cruzada
    verbose=2,
    random_state=random_state,
    n_jobs=-1  # Usar todos los núcleos disponibles
)

# Ajustar el modelo a los datos de entrenamiento
random_search.fit(X_train_encoded, y_train)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(random_search.best_params_)

# Predecir en el conjunto de validación con el mejor modelo encontrado
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_val_encoded)[:, 1]

# Calcular el AUC en el conjunto de validación
auc = roc_auc_score(y_val, y_pred_proba)
print(f"AUC en el conjunto de validación: {auc:.4f}")

### Hyperopt

In [ ]:
# Definir el espacio de búsqueda de hiperparámetros
space = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'max_depth': hp.choice('max_depth', [None, 10, 20, 30, 40, 50]),
    'max_samples': hp.uniform('max_samples', 0.5, 1.0),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10]),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4]),
    'bootstrap': True
}

# Función objetivo para Hyperopt
def objective(params):
    # Asegurarse de que 'max_samples' solo se usa si 'bootstrap' es True
    if not params['bootstrap']:
        params['max_samples'] = None  # No se puede utilizar max_samples si bootstrap es False
    
    # Definir el modelo con los hiperparámetros actuales
    model = RandomForestClassifier(**params, random_state=random_state)
    
    # Entrenar el modelo directamente ya que el Target Encoder e Imputador ya se aplicaron
    model.fit(X_train_encoded, y_train)

    # Hacer predicciones en el conjunto de validación
    y_pred_proba = model.predict_proba(X_val_encoded)[:, 1]

    # Calcular el AUC en el conjunto de validación
    auc = roc_auc_score(y_val, y_pred_proba)
    
    # Imprimir los hiperparámetros y el AUC actual
    print(f"Hiperparámetros: {params}, AUC: {auc:.4f}")

    # Retornar el valor negativo del AUC ya que Hyperopt minimiza por defecto
    return {'loss': 1-auc, 'status': STATUS_OK}

# Ejecutar la optimización
trials = Trials()  # Para guardar información sobre cada iteración
best = fmin(fn=objective, 
            space=space, 
            algo=tpe.suggest, 
            max_evals=50,  # Número de evaluaciones
            trials=trials)

print("Mejores hiperparámetros encontrados:")
print(best)

## XGBoost

### Hyperopt

In [18]:
# Espacio de búsqueda para los hiperparámetros
space = {
    'max_depth': hp.choice('max_depth', range(3, 10)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'min_child_weight': hp.uniform('min_child_weight', 1, 10),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 0.5, 2)
}

# Función objetivo para Hyperopt
def objective(params):
    # Entrenar el modelo con los hiperparámetros actuales
    model = xgb.XGBClassifier(
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        gamma=params['gamma'],
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        scale_pos_weight=params['scale_pos_weight'],
        use_label_encoder=False,
        eval_metric='auc',
        random_state=random_state
    )
    
    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train_encoded, y_train)
    
    # Predecir las probabilidades para calcular el AUC en el conjunto de validación
    y_pred_proba = model.predict_proba(X_val_encoded)[:, 1]
    auc = roc_auc_score(y_val, y_pred_proba)
    
    # Imprimir los hiperparámetros y el AUC actual
    print(f"Hiperparámetros: {params}, AUC: {auc:.4f}")
    
    return {'loss': -auc, 'status': STATUS_OK}

# Ejecutar la optimización
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,  # Número de evaluaciones
            trials=trials)

print("Mejores hiperparámetros encontrados:")
print(best)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instea

Hiperparámetros: {'colsample_bytree': 0.6189520886008961, 'gamma': 0.05263130692482787, 'learning_rate': 0.016434700376417533, 'max_depth': 3, 'min_child_weight': 6.378545393104531, 'n_estimators': 200, 'scale_pos_weight': 1.9885767576831004, 'subsample': 0.6696027253148403}, AUC: 0.8376
  2%|▏         | 1/50 [01:58<1:36:47, 118.51s/trial, best loss: -0.8375552417610834]

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version

  2%|▏         | 1/50 [12:15<10:00:48, 735.69s/trial, best loss: -0.8375552417610834]


KeyboardInterrupt: 

In [ ]:
# Reentrenar el modelo con los mejores hiperparámetros
best_model = xgb.XGBClassifier(
    max_depth=int(best['max_depth']),
    learning_rate=best['learning_rate'],
    n_estimators=int(best['n_estimators']),
    gamma=best['gamma'],
    min_child_weight=best['min_child_weight'],
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    scale_pos_weight=best['scale_pos_weight'],
    use_label_encoder=False,
    eval_metric='auc',
    random_state=random_state
)

# Entrenar el modelo con los mejores hiperparámetros en los datos completos
best_model.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de validación
y_pred_proba_val = best_model.predict_proba(X_val)[:, 1]
auc_val = roc_auc_score(y_val, y_pred_proba_val)
print(f"AUC en el conjunto de validación: {auc_val:.4f}")

## Pueba de Hiperparametros

### Random Forest

#### A mano

In [ ]:
# Reconstruir el modelo con los mejores hiperparámetros
best_model_rf = RandomForestClassifier(
    n_estimators=500,
    max_depth=20,
    max_samples=None,
    min_samples_split=5,
    min_samples_leaf=4,
    bootstrap=False,
    random_state=random_state
)

# Crear el pipeline con el mejor modelo
rf_pipeline = make_pipeline(SimpleImputer(), best_model_rf)

# Entrenar el modelo con los datos de entrenamiento
rf_pipeline.fit(X_train_encoded, y_train)

In [ ]:
# Predecir en el conjunto de testeo
y_preds_rf_test = rf_pipeline.predict_proba(test_data_encoded)[:, 1]

# Crear el archivo de envío
submission_df_rf = pd.DataFrame({"id": test_data["id"], "Label": y_preds_rf_test})
submission_df_rf["id"] = submission_df_rf["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "random_forest"
file_name_rf = (
    f"random_forest_predictions_n_estimators_{500}_"
    f"max_depth_{20}_"
    f"min_samples_split_{5}_"
    f"min_samples_leaf_{4}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_rf.to_csv(os.path.join("submits", file_name_rf), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_rf}")

#### Automatizada

In [ ]:
# Convertir los hiperparámetros a enteros
best['n_estimators'] = int(best['n_estimators'])
best['min_samples_split'] = int(best['min_samples_split'])
best['min_samples_leaf'] = int(best['min_samples_leaf'])

# Reconstruir el modelo con los mejores hiperparámetros
best_model_rf = RandomForestClassifier(
    n_estimators=best['n_estimators'],
    max_depth=[None, 10, 20, 30, 40, 50][best['max_depth']],  # Hyperopt devuelve índices para opciones de lista
    max_samples=best['max_samples'],  # Para max_samples no es un índice, es un valor continuo
    min_samples_split=best['min_samples_split'],
    min_samples_leaf=best['min_samples_leaf'],
    bootstrap=[True, False][best['bootstrap']],  # Igual que max_depth
    random_state=random_state
)

# Crear el pipeline con el mejor modelo
rf_pipeline = make_pipeline(SimpleImputer(), best_model_rf)

# Entrenar el modelo con los datos de entrenamiento
rf_pipeline.fit(X_train_encoded, y_train)

# Predecir en el conjunto de testeo
test_data_encoded = target_encoder.transform(test_data.select_dtypes(include='number').drop(columns=["id"]))
test_data_encoded = imputer.transform(test_data_encoded)  # Imputar los valores faltantes en el set de testeo
y_preds_rf_test = rf_pipeline.predict_proba(test_data_encoded)[:, 1]

# Crear el archivo de envío
submission_df_rf = pd.DataFrame({"id": test_data["id"], "Label": y_preds_rf_test})
submission_df_rf["id"] = submission_df_rf["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "random_forest"
file_name_rf = (
    f"random_forest_predictions_n_estimators_{best['n_estimators']}_"
    f"max_depth_{best['max_depth']}_"
    f"min_samples_split_{best['min_samples_split']}_"
    f"min_samples_leaf_{best['min_samples_leaf']}_"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_rf.to_csv(os.path.join("submits", file_name_rf), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_rf}")


### XGboost

#### A mano

In [ ]:
# Reconstruir el modelo de XGBoost con los mejores hiperparámetros
best_model_xgb = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=20,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=4,
    gamma=0.1,
    random_state=random_state,
    use_label_encoder=False,  # Si estás usando una versión reciente de XGBoost
    eval_metric="auc"  # Ajustar el métrico de evaluación
)

# Crear el pipeline con el mejor modelo
xgb_pipeline = make_pipeline(SimpleImputer(), best_model_xgb)

# Entrenar el modelo con los datos de entrenamiento
xgb_pipeline.fit(X_train_encoded, y_train)

# Predecir en el conjunto de testeo
y_preds_xgb_test = xgb_pipeline.predict_proba(test_data_encoded)[:, 1]

# Crear el archivo de envío
submission_df_xgb = pd.DataFrame({"id": test_data["id"], "Label": y_preds_xgb_test})
submission_df_xgb["id"] = submission_df_xgb["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "xgboost"
file_name_xgb = (
    f"xgboost_predictions_n_estimators_{500}_"
    f"max_depth_{20}_"
    f"learning_rate_{0.05}_"
    f"min_child_weight_{4}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_xgb.to_csv(os.path.join("submits", file_name_xgb), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_xgb}")


#### Atomatizada

In [ ]:
# Convertir los hiperparámetros a enteros si es necesario
best['n_estimators'] = int(best['n_estimators'])
best['max_depth'] = [None, 10, 20, 30, 40, 50][best['max_depth']]  # Ajustar el valor del índice si es necesario
best['min_child_weight'] = int(best['min_child_weight'])

# Reconstruir el modelo de XGBoost con los mejores hiperparámetros
best_model_xgb = xgb.XGBClassifier(
    n_estimators=best['n_estimators'],
    max_depth=best['max_depth'],
    learning_rate=best['learning_rate'],
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    min_child_weight=best['min_child_weight'],
    gamma=best['gamma'],
    random_state=random_state,
    use_label_encoder=False,
    eval_metric="auc"  # Establecer AUC como métrica de evaluación
)

# Crear el pipeline con el mejor modelo
xgb_pipeline = make_pipeline(SimpleImputer(), best_model_xgb)

# Entrenar el modelo con los datos de entrenamiento
xgb_pipeline.fit(X_train_encoded, y_train)

# Predecir en el conjunto de testeo
test_data_encoded = target_encoder.transform(test_data.select_dtypes(include='number').drop(columns=["id"]))
test_data_encoded = imputer.transform(test_data_encoded)  # Imputar los valores faltantes en el set de testeo
y_preds_xgb_test = xgb_pipeline.predict_proba(test_data_encoded)[:, 1]

# Crear el archivo de envío
submission_df_xgb = pd.DataFrame({"id": test_data["id"], "Label": y_preds_xgb_test})
submission_df_xgb["id"] = submission_df_xgb["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "xgboost"
file_name_xgb = (
    f"xgboost_predictions_n_estimators_{best['n_estimators']}_"
    f"max_depth_{best['max_depth']}_"
    f"learning_rate_{best['learning_rate']}_"
    f"min_child_weight_{best['min_child_weight']}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_xgb.to_csv(os.path.join("submits", file_name_xgb), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_xgb}")
